# Initial Sizing

This section performs initial weight estimation for the example aircraft. The maximum takeoff weight ($W_{\text{MTO}}$) can be divied into
three components for this initial estimation: payload ($W_{p}$), empty weight ($W_e$), and fuel weight ($W_f$). Mathematically, 
it can be written as

$$
    W_{\text{MTO}} = W_p + W_e + W_f.
$$

Note that $W_p$ includes crew weight, along with passenger and baggage weight. Following subsections demonstrates methods for 
computing $W_e$, $W_f$ and MTOW.

## Empty Weight Fraction

During the intial phase of design, the empty weight is usually computed based on the historical data for similar type of aircraft.
Most aircraft design books will provide some sort of empirical formula to compute $W_e$ or empty weight fraction ($W_e/W_f$). In this
demonstration, empty weight trends provided in Nicolai and Carichner's book are used. The empty weight fraction is computed using

$$
    W_e/W_{\text{MTO}} = 0.911 \times W_{\text{MTO}} ^ {0.947}.
$$

> **_NOTE:_** In the above formula, MTOW should be in pounds (lbs)

Following code defines a simple python function for computing $W_e/W_f$ which is later used for computing MTOW.

In [ ]:
def compute_empty_weight_fraction(MTOW):
    
    # empty weight fraction (from Nicolai and Carichner, Pg. 763)
    We_by_MTOW = 0.911 * MTOW ** 0.947
    
    return We_by_MTOW

## Fuel Fraction

The fuel fraction is computed using mission segment weight fraction method. In this method, 

Breguet Range equation for a propeller aircraft (piston/turboprop) is given by

$$
    R = \frac{\eta_{cr}}{C} \Big(\frac{L}{D}\Big)_{cr} \ln\Bigg(\frac{W_{i-1}}{W_i}\Bigg)
$$

$$
    \frac{W_i}{W_{i-1}} = \exp\Bigg( \frac{-RC}{V(L/D)} \Bigg)
$$

$$
    T = \frac{\eta}{C}(L/D)_{loiter}\frac{1}{V}\ln\Bigg( \frac{W_{i-1}}{W_i} \Bigg)
$$

For a propeller powered airplane

most efficient cruise velocity occurs at approx $(L/D)_{max}$

most efficient loiter velocity occurs at approx $0.866 \times (L/D)_{max}$

Propeller aircraft loiter equation is given by (eq 5.7, pg. 131 Nicolai)

Next step is to estimate maximum L/D

$$
    (L/D)_{max} = 
$$

Next step is to estimate zero-lift drag ($C_{D_0}$)


In [ ]:
def compute_CD0(MTOW):

    pass

def compute_fuel_fraction(MTOW):

    # variables
    range = 1200 # nmi
    V_cruise = 250 # KTAS


    # Table 2.1 Roskam Part 1 (Pg. 12)

    W1_by_MTOW = 0.984 # warmup, taxi, takeoff

    W2_by_W1 = 0.99 # climb

    W3_by_W2 = # cruise

    W4_by_W3 = 0.992 # descent

    W5_by_W4 = 0.99 # climb

    W6_by_W5 =  # loiter

    W7_by_W6 = 0.992 # descent

    W8_by_W7 = 0.992 # land, taxi, shutdown

    W8_by_MTOW = W1_by_MTOW * W2_by_W1 * W3_by_W2 * W4_by_W3 \
                * W5_by_W4 * W6_by_W5 * W7_by_W6 * W8_by_W7

    Wf_by_MTOW = 1.06 * (1 - W8_by_MTOW) # 6% is for trapped unusable fuel

    return Wf_by_MTOW

## MTOW Estimation

$$
    W_{\text{MTO}} = \frac{W_p}{1 - \dfrac{W_e}{W_{\text{MTO}}} - \dfrac{W_f}{W_{\text{MTO}}}}
$$